In [19]:
import pandas as pd
import numpy as np
from cyvcf2 import VCF

In [20]:
pemb = pd.read_csv("/storage/s1saini/manuscript_strsnp/fig3/cap_elec/Pemberton_AdditionalFile1_11242009.txt", delim_whitespace=True)
onekg_snp_info = pd.read_csv("/storage/s1saini/manuscript_strsnp/fig3/cap_elec/STRPinfo.txt", delim_whitespace=True)
locus_ref = pd.merge(pemb, onekg_snp_info, how="inner", left_on="alternateName", right_on="MarkerName")[["alternateName", "lengthRefSeq(bp)", "expectedPCRfragmentSize_forRefSeq(bp)", "Chrom", "Bpposition", "Repeattype"]]
locus_ref.columns = ['id', 'reflen1', 'reflen2', 'chrom', 'pos', 'type']

In [21]:
# load the genotype data
gtData = pd.read_csv("/storage/s1saini/manuscript_strsnp/fig3/cap_elec/STRPgenotypes.txt", delim_whitespace=True)
samples = [x for x in list(gtData.columns.values) if "NA" in x]
gtData = gtData.merge(locus_ref, how="inner", left_on="Name", right_on="id")
gtData_converted = gtData.copy()

In [22]:
# round to the nearest repeat unit length
type_to_length = {'Di':2.0, 'Tetra':4.0, 'Tri':3.0, 'Penta':5.0}
for i in range(gtData.shape[0]):
    repLength = type_to_length[gtData.at[i, 'type']]
    for sample in samples:
        gt = gtData.at[i, sample]
        if gt == "0/0":
            gtData_converted.at[i, sample] = "./."
        else:
            gt_new = [int(x)-gtData.at[i,"reflen1"] for x in gt.split("/")]
            gt_rounded = [int(np.floor(x/repLength)*repLength) for x in gt_new]
            # Filter if we only had to round 1 allele
            if gt_new[0]!= gt_rounded[0] and gt_new[1]!= gt_rounded[1]:
                gtData_converted.at[i, sample] = "/".join(map(str,gt_rounded))
            elif gt_new[0]== gt_rounded[0] and gt_new[1]== gt_rounded[1]:
                gtData_converted.at[i, sample] = "/".join(map(str,gt_rounded))
            else:
                gtData_converted.at[i, sample] = "./."            # Filter if not multiple of repeat unit
            #if gt_new[0]%repLength != 0 or gt_new[1]%repLength !=0:
            #    gtData_converted.at[i, sample] = "./." 
            #else:
            #    gtData_converted.at[i, sample] = "/".join(map(str,gt_new))

In [23]:
# load imputation data for the samples available in Marshfield set
myDF = pd.DataFrame()
chrom_bak = 0
for i in range(gtData.shape[0]):
    chrom = gtData.at[i,'chrom']
    ID = gtData.at[i,'Name']
    if chrom != chrom_bak:
        strReg = pd.read_csv("/storage/s1saini/str-imputation/hipstr_template/str_regions_bed/HipSTR.chr"+str(chrom)+".txt", delim_whitespace=True, names=['chrom', 'start', 'end', 'type', 'reflen', 'ID'])
    strRegPos = strReg[strReg['ID']==ID]['start']
    if strRegPos.shape[0] == 0:
        continue
    position = int(strRegPos)
    fname = '/storage/s1saini/manuscript_strsnp/fig3/1kg.panel.anno/1kg.snp.str.chr'+str(chrom)+'.vcf.gz'
    vcf = VCF(fname)
    for variant in vcf(str(chrom)+":"+str(position)+"-"+str(position)):
        if variant.ID == ID:
            tmp1 = [x.split("|") for x in variant.gt_bases]
            gt = ["/".join(map(str,[len(x)-len(variant.REF) for x in gt])) for gt in tmp1]
            myDF = myDF.append(pd.DataFrame([dict(zip(["Name"]+vcf.samples, [variant.ID]+gt))], columns=dict(zip(["Name"]+vcf.samples, [variant.ID]+gt)).keys()), ignore_index=True)
            break
    chrom_bak = chrom

/usr/lib/python3.4/site-packages/ipykernel/__main__.py:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


In [24]:
common_samples = set([x for x in list(gtData_converted.columns.values) if "NA" in x]).intersection(set([x for x in list(myDF.columns.values) if "NA" in x]))

In [25]:
mergedData = pd.DataFrame()
for i in range(gtData_converted.shape[0]):
    for sample in common_samples:
        gral1, gral2 = gtData_converted.at[i, sample].split("/")
        if myDF[myDF['Name']==gtData_converted.at[i,"Name"]].shape[0] != 0:
            imal1, imal2 = myDF[myDF['Name']==gtData.at[i,"Name"]][sample].values[0].split("/")
            mergedData = mergedData.append(pd.DataFrame([{'name':gtData.at[i,"Name"], 'sample':sample, 'gral1':gral1, 'gral2':gral2, 'imal1':imal1, 'imal2':imal2}]))

In [26]:
mergedData.to_csv("capillary_vs_imputed_calls.csv", index=False, columns=['sample', 'name', 'gral1', 'gral2', 'imal1', 'imal2'])